In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [15]:
image_dir = Path('./age_prediction/')

In [16]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath',dtype=str)
ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(int)

images = pd.concat([filepaths, ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [18]:
images = images.rename(columns={'Filepath': 'Tensor'})
images

,Tensor,Age
0,age_prediction/train/036/5850.jpg,36
1,age_prediction/train/041/152701.jpg,41
2,age_prediction/test/049/3635.jpg,49
3,age_prediction/train/026/1302.jpg,26
4,age_prediction/train/034/102056.jpg,34
...,...,...
233195,age_prediction/train/051/104882.jpg,51
233196,age_prediction/test/048/16922.jpg,48
233197,age_prediction/train/059/148335.jpg,59
233198,age_prediction/train/067/83603.jpg,67


In [19]:
# Let's only use 500 images to speed up training time
image_df = images.sample(100, random_state=1).reset_index(drop=True)

# train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [20]:
# Define image size
IMG_SIZE = 224

# Create a function for preprocessing images
def process_image(image_path, img_size=IMG_SIZE):
  """
  Takes an image path and turns the image into a Tensor
  """
  # Read in an image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the color channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired value (224,224)
  image = tf.image.resize(image, size=[img_size, img_size])

  return image

In [21]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

In [22]:
def func(row):
  return process_image(row[0])

image_df['Tensor'] = image_df.apply(func,axis=1)

In [23]:
image_df.head()

,Tensor,Age
0,"(((tf.Tensor(0.34117648, shape=(), dtype=float...",30
1,"(((tf.Tensor(0.94117653, shape=(), dtype=float...",44
2,"(((tf.Tensor(0.45098042, shape=(), dtype=float...",36
3,"(((tf.Tensor(0.95294124, shape=(), dtype=float...",69
4,"(((tf.Tensor(0.7686275, shape=(), dtype=float3...",36


## Noah Testing

## Zach Testing